In [ ]:
def fill_missing_values(data, columns_to_fill, corr_matrix):
    for column in columns_to_fill:
        if column in corr_matrix.columns:
            most_corr_features = corr_matrix[column][corr_matrix[column].abs() > 0.1].index.tolist()
            if column in most_corr_features:
                most_corr_features.remove(column)

            if most_corr_features:
                group_modes = data.groupby(most_corr_features)[column].apply(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

                def fill_na(row):
                    if pd.isna(row[column]):
                        try:
                            return group_modes.loc[tuple(row[most_corr_features])]
                        except KeyError:
                            return np.nan
                    else:
                        return row[column]

                data[column] = data.apply(fill_na, axis=1)

fill_missing_values(df_encoded, columns_to_fill_train, corr_matrix_train)
fill_missing_values(dt_encoded, columns_to_fill_test, corr_matrix_test)


In [ ]:
def cleaning(df):
    threshold = 100
    
    cat_feats = ['cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-root', 'stem-surface', 'stem-color', 'spore-print-color',
       'habitat', 'season', 'veil-info', 'has-ring-type']
    
    for feat in cat_feats:
        if df[feat].dtype.name == 'category':
            # Add 'missing' and 'noise' to categories if not present
            if 'missing' not in df[feat].cat.categories:
                df[feat] = df[feat].cat.add_categories('missing')
            if 'noise' not in df[feat].cat.categories:
                df[feat] = df[feat].cat.add_categories('noise')
        else:
            # Convert to category and add new categories
            df[feat] = df[feat].astype('category')
            df[feat] = df[feat].cat.add_categories(['missing', 'noise'])
        
        # Fill missing values with 'missing'
        df[feat] = df[feat].fillna('missing')
        
        # Replace infrequent categories with 'noise'
        counts = df[feat].value_counts(dropna=False)
        infrequent_categories = counts[counts < threshold].index
        df[feat] = df[feat].apply(lambda x: 'missing' if x in infrequent_categories else x)
    
    return df